# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import functools
import spatialdata
import spatialdata_io as sdio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import corescpy as cr

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.

# Setup

## Options & Data

In [2]:
%%time

# Count Threshold for Cell Quantification
count_threshold = 1

# File Paths
libid = "Uninflamed-50336C"
dir_data = "/mnt/cho_lab/bbdata2/outputs/TUQ97N"
out_dir = str("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/"
              "outputs/TUQ97N/nebraska")
path_dir = os.path.join(out_dir, "pathology")
file_align = os.path.join(path_dir,
                          f"alignment/{libid}_alignment_files/matrix.csv")
file_image = (os.path.join(path_dir, f"{libid.split('-')[1]}.ndpi"),  # raw
              os.path.join(path_dir, f"ome-tiff/{libid}.ome.tif"))  # convert

# Clustering Version
col_cell_type = "leiden_res1pt5_dist0_npc30"  # high resolution
# c_t = "leiden_res0pt75_dist0pt3_npc30"  # medium resolution
# c_t = "leiden_res0pt5_dist0pt5_npc30"  # low resolution

# Display
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500

# Spatial Data
files = functools.reduce(lambda i, j: i + j, [[os.path.join(
    run, i) for i in os.listdir(os.path.join(
        dir_data, run))] for run in os.listdir(dir_data)])
file_path = np.array(files)[np.where(["-".join(libid.split(
    "-")[1:]) == os.path.basename(x).split("__")[2].split(
        "-")[0] for x in files])[0][0]]
self = cr.Spatial(os.path.join(dir_data, file_path), library_id=libid,
                  col_cell_type=col_cell_type, n_jobs=16)
self.update_from_h5ad(os.path.join(out_dir, libid + ".h5ad"))
self.get_layer("counts", inplace=True)
self.adata



<<< INITIALIZING SPATIAL CLASS OBJECT >>>

INFO     reading                                                                                                   
         /mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-010/output-XETG00189__0011047__50336C-TUQ97N-EA__20240422__175051/
         cell_feature_matrix.h5                                                                                    


Counts: Initial


	Observations: 387961

	Genes: 469







 AnnData object with n_obs × n_vars = 387961 × 469
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region', 'z_level', 'nucleus_count', 'cell_labels', 'Sample'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs', 'spatial', 'original_ix'
    obsm: 'spatial'
    layers: 'counts' 

                      gene_ids    feature_types   genome
gene_symbols                     

SpatialData object with:
├── Images
│     └── 'morphology_focus': MultiscaleSpatialImage[cyx] (5, 74826, 48440), (5, 37413, 24220), (5, 18706, 12110), (5, 9353, 6055), (5, 4676, 3027)
├── Labels
│     ├── 'cell_labels': MultiscaleSpatialImage[yx] (74826, 48440), (37413, 24220), (18706, 12110), (9353, 6055), (4676, 3027)
│     └── 'nucleus_labels': MultiscaleSpatialImage[yx] (74826, 48440), (37413, 24220), (18706, 12110), (9353, 6055), (4676, 3027)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 11) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (387961, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (387961, 2) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (379551, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (378219, 469)
with coordinate systems:
▸ 'global', with elements:
        morphology_focus (Images), cell_labels (Labels), nucleus_labels (Labels), transcripts (Points), cell_boundaries (Shapes)

## Load Annotations

In [ ]:
fmr = os.path.join(out_dir, "annotation_dictionaries", str(
    f"{self._library_id}___{col_cell_type}_dictionary.xlsx"))  # file
fmr = pd.read_excel(fmr).astype(str)
c_m = col_cell_type.split("leiden_")[1]
for x in ["annotation", "bin", "bucket"]:
    self.rna.obs.loc[:, f"{x}_{c_m}"] = self.rna.obs[col_cell_type].astype(
        int).astype(str).replace(fmr.set_index(fmr.columns[0])[x])  # to label
    self.rna.obs.loc[self.rna.obs[f"{x}_{c_m}"].isnull(
        ), f"{x}_{c_m}"] = self.rna.obs.loc[self.rna.obs[f"{x}_{c_m}"].isnull(
            ), col_cell_type].astype(str)  # missing annotations -> Leiden
    self.rna.obs.loc[:, f"{x}_{c_m}"] = self.rna.obs[
        f"{x}_{c_m}"].astype("category")  # as categorical
    # self.plot_spatial(f"{x}_{r}")

## Load Image

In [ ]:
if os.path.exists(file_image[1]) is False:  # convert to ome-tiff if needed
    cr.tl.write_ome_tif(file_image[0], file_image[1],
                        bf_cmd="cd ~/bftools && ")
self.add_image(file_image[1], name="he", file_align=file_align)  # add image

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# from napari_spatialdata import Interactive
# from spatialdata import SpatialData

# plt.rcParams["figure.figsize"] = (20, 20)

# sdata = self.adata
# interactive = Interactive(sdata)
# interactive.run()

# Render Images

In [ ]:
axes = plt.subplots(3, 1, figsize=(20, 13))[1].flatten()
self.adata.pl.render_images("he").pl.show(
    ax=axes[0], title="H&E", coordinate_systems="global")
self.adata.pl.render_images("morphology_focus").pl.show(
    ax=axes[1], title="Morphology", coordinate_systems="global")
self.adata.pl.render_shapes(color=col_cell_type).pl.show(
    ax=axes[2], title="Labels", coordinate_systems="global")

In [ ]:
self.adata.pl.render_images("morphology_focus").pl.show(
    title="Morphology", coordinate_systems="global", dpi=20)
self.adata.pl.render_shapes(elements="cell_boundaries", groups=[
    "1", "2"], color=col_cell_type).pl.show(coordinate_systems="global")

In [ ]:
axes = plt.subplots(1, 2, figsize=(30, 30))[1].flatten()
self.adata.pl.render_images("he").pl.show(
    ax=axes[0], title="H&E", coordinate_systems="global")
self.adata.pl.render_shapes(color=col_cell_type).pl.show(
    ax=axes[1], title="Labels", coordinate_systems="global")

In [16]:
self.adata

SpatialData object with:
├── Images
│     ├── 'he': SpatialImage[cyx] (3, 59136, 84480)
│     └── 'morphology_focus': MultiscaleSpatialImage[cyx] (5, 74826, 48440), (5, 37413, 24220), (5, 18706, 12110), (5, 9353, 6055), (5, 4676, 3027)
├── Labels
│     ├── 'cell_labels': MultiscaleSpatialImage[yx] (74826, 48440), (37413, 24220), (18706, 12110), (9353, 6055), (4676, 3027)
│     └── 'nucleus_labels': MultiscaleSpatialImage[yx] (74826, 48440), (37413, 24220), (18706, 12110), (9353, 6055), (4676, 3027)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 11) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (387961, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (387961, 2) (2D shapes)
│     └── 'nucleus_boundaries': GeoDataFrame shape: (379551, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (378219, 469)
with coordinate systems:
▸ 'global', with elements:
        he (Images), morphology_focus (Images), cell_labels (Labels), nucleu

In [19]:
sdata = self.crop([800, 1000], [600, 800])

In [20]:
sdata

SpatialData object with:
├── Images
│     ├── 'he': SpatialImage[cyx] (3, 193, 193)
│     └── 'morphology_focus': MultiscaleSpatialImage[cyx] (5, 200, 200), (5, 100, 100), (5, 50, 50), (5, 25, 25), (5, 13, 12)
└── Labels
      ├── 'cell_labels': MultiscaleSpatialImage[yx] (200, 200), (100, 100), (50, 50), (25, 25), (13, 12)
      └── 'nucleus_labels': MultiscaleSpatialImage[yx] (200, 200), (100, 100), (50, 50), (25, 25), (13, 12)
with coordinate systems:
▸ 'global', with elements:
        he (Images), morphology_focus (Images), cell_labels (Labels), nucleus_labels (Labels)

In [26]:
self.adata = sdata

In [ ]:
sdata.pl.render_images("morphology_focus", scale="scale4").pl.show(
    title="Morphology", coordinate_systems="global")

# Image Analysis

In [ ]:
sdata